In [1]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba

10:11:57+0000 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


# Graph Traversal

In [2]:
bd.projects.set_current("ecoinvent-3.12-cutoff")

In [3]:
import bw_graph_tools as bgt

In [4]:
bgt.GraphTraversalSettings?

Init signature:
bgt.GraphTraversalSettings(
    *,
    cutoff: Annotated[float, Strict(strict=True), Gt(gt=0), Lt(lt=1)] = 0.005,
    biosphere_cutoff: Annotated[float, Strict(strict=True), Gt(gt=0), Lt(lt=1)] = 0.0001,
    max_calc: Annotated[int, Strict(strict=True), Gt(gt=0)] = 1000,
    max_depth: Optional[int] = None,
    skip_coproducts: bool = False,
    separate_biosphere_flows: bool = True,
) -> None
Docstring:     
Graph traversal settings object with reasonable defaults.

Parameters
----------
cutoff : float
    Cutoff value used to stop graph traversal. Fraction of total score,
    should be in `(0, 1)`
biosphere_cutoff : float
    Cutoff value used to determine if a separate biosphere node is
    added. Fraction of total score.
max_calc : int | None
    Maximum number of inventory calculations to perform
max_depth : int
    Maximum depth in the supply chain traversal. Default is no maximum.
skip_coproducts : bool
    Don't traverse co-production edges, i.e. production edge

In [5]:
settings = bgt.GraphTraversalSettings(max_calc=500)

In [6]:
node = bd.get_node(name="market for photovoltaic laminate, CIS", database="ecoinvent-3.12-cutoff")

In [7]:
random_ic = (
    'ecoinvent-3.12',
    'ReCiPe 2016 v1.03, endpoint (I) no LT',
    'total: human health no LT',
    'human health no LT'
)

In [8]:
fu, data_objs, _ = bd.prepare_lca_inputs({node: 1}, method=random_ic, remapping=False)

In [9]:
lca = bc.LCA(fu, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

6.494229717246775e-05

In [10]:
gt = bgt.NewNodeEachVisitGraphTraversal(
    lca=lca,
    settings=settings,
)

In [11]:
gt.traverse()

In [12]:
len(gt.nodes), len(gt.edges), len(gt.flows)

(172, 171, 89)

In [13]:
gt.nodes[0]

Node(unique_id=0, activity_datapackage_id=246938910078660612, activity_index=12771, reference_product_datapackage_id=246938910078660612, reference_product_index=12771, reference_product_production_amount=1.0, depth=1, supply_amount=1.0, cumulative_score=6.494229717246786e-05, direct_emissions_score=0.0, max_depth=None, direct_emissions_score_outside_specific_flows=0.0, remaining_cumulative_score_outside_specific_flows=6.494229717246786e-05, terminal=False)

In [14]:
gt.flows[0]

Flow(flow_datapackage_id=246938525838471172, flow_index=2529, activity_unique_id=35, activity_id=246938937761067008, activity_index=19064, amount=0.01422118847677928, score=8.94512778637504e-06)

# Small fixes

## Message when deleting project

In [15]:
bd.projects.delete_project("Bicycle example")

10:13:07+0000 [warning  ] Removing project from project Bicycle example list, but not deleting data; if you switch to this project again you will have the same data again. To delete data permanently, pass `(..., delete_dir=True)`.


'ecoinvent-3.12-cutoff'

In [16]:
bd.projects.set_current("Bicycle example")

In [17]:
bd.databases

Databases dictionary with 3 object(s):
	better bike
	🚲
	🚲 🚬 😔

## Edge display string

In [18]:
cf = bd.get_node(database='🚲', name='carbon fibre production')
for edge in cf.exchanges():
    print(edge)

Exchange: 1 kilogram 'carbon fibre' (kilogram, GLO, None) from 'carbon fibre production' (None, DE, None)
Exchange: 237.3 megajoule 'natural gas' (megajoule, GLO, None) to 'carbon fibre production' (None, DE, None)
Exchange: 26.6 kilogram 'Carbon Dioxide' (kilogram, GLO, ('air',)) to 'carbon fibre production' (None, DE, None)


# Extracting a subgraph from ecoinvent to a new database

In [19]:
bd.projects.set_current("ecoinvent-3.12-cutoff")

In [20]:
db = bd.Database("ecoinvent-3.12-cutoff")

In [21]:
laminate = bd.get_node(name="market for photovoltaic laminate, CIS", database="ecoinvent-3.12-cutoff")
ba.print_recursive_supply_chain(laminate, max_level=1)

1: 'market for photovoltaic laminate, CIS' (square meter, GLO, None)
  0.64: 'photovoltaic laminate production, CIS' (square meter, RoW, None)
  0.36: 'photovoltaic laminate production, CIS' (square meter, DE, None)


In [22]:
panel = bd.get_node(name="market for photovoltaic panel, CIS", database="ecoinvent-3.12-cutoff")
ba.print_recursive_supply_chain(panel, max_level=1)

1: 'market for photovoltaic panel, CIS' (square meter, GLO, None)
  0.64: 'photovoltaic panel production, CIS' (square meter, RoW, None)
  0.36: 'photovoltaic panel production, CIS' (square meter, DE, None)


In [23]:
nodes = [laminate, panel] + [exc.input for exc in laminate.technosphere()] + [exc.input for exc in panel.technosphere()]
nodes

['market for photovoltaic laminate, CIS' (square meter, GLO, None),
 'market for photovoltaic panel, CIS' (square meter, GLO, None),
 'photovoltaic laminate production, CIS' (square meter, RoW, None),
 'photovoltaic laminate production, CIS' (square meter, DE, None),
 'photovoltaic panel production, CIS' (square meter, RoW, None),
 'photovoltaic panel production, CIS' (square meter, DE, None)]

In [24]:
subgraph = bd.Database("PV production")
subgraph.register()

In [25]:
new_nodes = db.copy_activities(nodes, "PV production")

In [26]:
list(subgraph)

['market for photovoltaic laminate, CIS' (square meter, GLO, None),
 'photovoltaic laminate production, CIS' (square meter, DE, None),
 'market for photovoltaic panel, CIS' (square meter, GLO, None),
 'photovoltaic laminate production, CIS' (square meter, RoW, None),
 'photovoltaic panel production, CIS' (square meter, RoW, None),
 'photovoltaic panel production, CIS' (square meter, DE, None)]

In [27]:
new_laminate = bd.get_node(name='market for photovoltaic laminate, CIS', database="PV production")

In [28]:
for exc in new_laminate.technosphere():
    print(exc.input["database"], exc)

PV production Exchange: 0.639999999999999 square meter 'photovoltaic laminate production, CIS' (square meter, RoW, None) to 'market for photovoltaic laminate, CIS' (square meter, GLO, None)
PV production Exchange: 0.360000000000001 square meter 'photovoltaic laminate production, CIS' (square meter, DE, None) to 'market for photovoltaic laminate, CIS' (square meter, GLO, None)


# Creating aggregated processes from unit processes

We can do this by calculating the inventory and reading the non-zero elementary flow amounts.

In [30]:
agg, _ = new_laminate.create_aggregated_process(database="PV production", name="CIS laminate, aggregated")

10:14:02+0000 [info     ] Changed code to avoid conflict with existing value: {self['code']} to {data['code']}
10:14:02+0000 [info     ] Successfully switch activity dataset to database `PV production`


In [31]:
len(agg.biosphere()), len(agg.technosphere()), len(agg.production())

(2747, 0, 1)